<a href="https://colab.research.google.com/github/tugangui/caseTracker/blob/main/caseTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
import pandas as pd
import numpy as np
import os

# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

# List the tab names. Could be used for asserting in next section
# titles_list = gc.open('Copy of Hongwei Dong lab case tracker.xlsx').worksheets()
# print(titles_list)

Mounted at /content/drive


In [2]:
# tab names and pivotal column (case column)
tab_names = {'U01 ':1,'U19 Salk Institute':1,'U19 CSHL':1,'MCP':1,'Basal Ganglia':1,'RF1 (HPF)':1,'BLA project':1}
surgeon_cols = {'U01 ':9,'U19 Salk Institute':7,'U19 CSHL':8,'MCP':8,'Basal Ganglia':7,'RF1 (HPF)':7,'BLA project':8}
tabs = {}
for k,v in tab_names.items():
  worksheet = gc.open('Copy of Hongwei Dong lab case tracker.xlsx').worksheet(k)
  tabs[k] = pd.DataFrame(worksheet.get_all_values())  

In [3]:
distinctCases = pd.DataFrame()
tracersPathways = pd.DataFrame()
actualInjections = pd.DataFrame()
total = pd.DataFrame()

i=0
for k,tab in tabs.items():  
  piv = tab_names[k]
  s = surgeon_cols[k]
  casesSurgeons = tab[[piv, s]].copy()
  casesSurgeons = casesSurgeons.rename(columns={piv: 'Case', s: 'Surgeon'})
  casesSurgeons = casesSurgeons.drop_duplicates(subset='Case', keep='first')
  casesSurgeons = casesSurgeons[casesSurgeons['Case'].str.startswith('S')]
  casesSurgeons['length']= (casesSurgeons['Case']).str.len()
  casesSurgeons = casesSurgeons[(casesSurgeons.length>=11) & (casesSurgeons.length<=13)] # some cases have asterisks?

  cases = pd.DataFrame(tab.iloc[:, piv].unique()) # Distinct cases  
  cases = cases.applymap(str)
  cases = cases.loc[cases[0].str.startswith('S')]
  cases['length']= (cases[0]).str.len()
  cases = cases[(cases.length>=11) & (cases.length<=13)] # some cases have asterisks?

  tracers = pd.DataFrame(tab.iloc[:, piv+2]) # total
  mapping = {tracers.columns[0]:'tracers'}
  tracers = tracers.rename(columns=mapping)
  tracers['length']= (tracers['tracers']).str.len()
  tracers = tracers[(tracers.length>1)] # filter out blank entries

  if(k=='BLA project' or k=='U01 '):
    injections = pd.DataFrame(tab.iloc[:, piv+4]) # Intended injections column is column 5
  else:
    injections = pd.DataFrame(tab.iloc[:, piv+3]) # total
  mapping = {injections.columns[0]:'injections'}
  injections = injections.rename(columns=mapping)
  injections['length']= (injections['injections']).str.len()
  injections = injections[(injections.length>1)] # filter out blank entries

  distinctCases = distinctCases.append(cases)
  tracersPathways = tracersPathways.append(tracers)
  actualInjections = actualInjections.append(injections)
  total = total.append(casesSurgeons)
  i += 1

total['Case'] = total['Case'].str.slice(stop=11)
total.to_csv('/total.csv')

tracersPathways = tracersPathways.dropna()
actualInjections = actualInjections.dropna()
distinctCases = distinctCases.dropna()

distinctCases.iloc[:,0] = distinctCases.iloc[:,0].str.slice(stop=11)
distinctCases = pd.DataFrame(distinctCases.iloc[:,0].unique())
distinctCases.index = distinctCases.index + 1
distinctCases.to_csv('/distinctCases.csv', header=False)
print('Distinct cases: '+ str(distinctCases.size))
print('/distinctCases.csv')

tracersPathways = tracersPathways.iloc[:, 0]
tracersPathways.index = tracersPathways.index + 1
pd.DataFrame(tracersPathways).to_csv('/tracersPathways.csv', header=False)
print('Total tracers: '+ str(tracersPathways.size))
print('/tracersPathways.csv')

actualInjections = actualInjections.iloc[:, 0]
actualInjections.index = actualInjections.index + 1
pd.DataFrame(actualInjections).to_csv('/actualInjections.csv', header=False)
print('Total injections: ' + str(actualInjections.size))
print('/actualInjections.csv')

ser = [distinctCases, tracersPathways, actualInjections]
my_names = ['Cases','Injections','Tracers']

df = pd.concat(ser, axis=0, keys=my_names)
df.to_csv('/allColumns.csv')
print('/allColumns.csv')

Distinct cases: 1895
/distinctCases.csv
Total tracers: 7595
/tracersPathways.csv
Total injections: 6696
/actualInjections.csv
/allColumns.csv


In [9]:
def get_organism_id(argument):
    switcher = {
        'SG': 7,
        'SP': 17,
        'SV': 18
    }
    return switcher.get(argument, 1)

cases = pd.Series(distinctCases.iloc[:,0])
f = open("/casetracker.sql", "w")

query = "INSERT IGNORE INTO organisms (id, species, strain, allele_type, gene_marker, description, code) VALUES (17, 'mouse', 'Parvalbumin-Cre', 'transgenic', 'Parvalbumin (PV)', 'These mice selectively express Cre recombinase in PV expressing cells', 'P');\n"
query = query + "INSERT IGNORE INTO organisms (id, species, strain, allele_type, gene_marker, description, code) VALUES (18, 'mouse', 'Vasoactive Intestinal Peptide-Cre', 'transgenic', 'Vasoactive Intestinal Peptide (VIP)', 'These mice selectively express Cre recombinase in VIP expressing cells', 'P');\n"
query = query + "INSERT INTO users(user_name, email, user_group, is_active) VALUES('Lei Gao', 'LeiGao@mednet.ucla.edu',4, 1 )ON DUPLICATE KEY UPDATE email='LeiGao@mednet.ucla.edu';\n"

#insert into animals + tissueDissections
for row in total.itertuples(index=False):
    casePrefix = get_organism_id(row[0][:2])
    query = query + "INSERT INTO animals (organism_id) SELECT " + str(casePrefix) + " FROM tissueDissections WHERE NOT EXISTS(SELECT NULL FROM tissueDissections td WHERE td.tissue_code='" + row[0] + "') LIMIT 1;\n"
    query = query + "INSERT IGNORE INTO tissueDissections (tissue_code, tissue_name, description, animal_id) VALUES ('" + row[0] +"', 'brain', 'brain', LAST_INSERT_ID());\n"

# insert into surgeries
#Lin, Lei, Marlene, Kevin
def usernames(argument):
    switcher = {
        'Monica': "Monica Song",
        'Lei': "Lei Gao",
        'Marlene': "Marlene Becerra"
    }
    return switcher.get(argument, "Lin Gou")

for row in total.itertuples(index=False):
    username = usernames(row[1].strip())
    query = query + "\nINSERT INTO surgeries (user_id, animal_id, tissue_dissection_id) SELECT (select id from users where user_name='" + username + "') as user_id, (select animal_id from tissueDissections where tissue_code='" + row[0] + "') as animal_id, (select id FROM tissueDissections WHERE tissue_code='" + row[0] + "') as tissue_code from surgeries WHERE NOT EXISTS(SELECT s.tissue_dissection_id FROM surgeries s INNER JOIN tissueDissections td on s.tissue_dissection_id=td.id WHERE td.tissue_code='" + row[0] + "') LIMIT 1;"

f.write(query)
f.close()

